# 第7课：时序差分方法（Temporal-Difference Methods）

---

## Part 1：例子：为什么只用当前一步也能学值？

我们设想一个例子：一个智能体在走迷宫，当前状态为 $s_t$，采取动作 $a_t$，观察到奖励 $r_{t+1}$ 和新状态 $s_{t+1}$。

目标：估计状态值函数 $V(s)$

### 蒙特卡洛的做法：

> 等到 episode 结束，累加所有未来回报 $G_t$ 再更新 $V(s_t)$

### 时序差分（TD）方法的做法：

> 用下一个状态的值 $V(s_{t+1})$ 来近似当前状态的回报

$$
V(s_t) \leftarrow V(s_t) + \alpha \left[ r_{t+1} + \gamma V(s_{t+1}) - V(s_t) \right]
$$

这就是 TD(0)，它只需要一步就能进行更新。

---

## Part 2：TD算法介绍（TD(0)）

### 目标：

> 在不依赖环境模型的情况下，对策略 $\pi$ 进行**在线状态值估计**

---

### TD(0)更新规则：

$$
\boxed{
V(s_t) \leftarrow V(s_t) + \alpha \left( r_{t+1} + \gamma V(s_{t+1}) - V(s_t) \right)
}
$$

* $r_{t+1} + \gamma V(s_{t+1})$：称为 **TD目标**
* 整个更新项：称为 **TD误差（TD Error）**，记作 $\delta_t$

---

### TD(0) 的特性：

| 特性      | 说明              |
| ------- | --------------- |
| 增量式     | 无需等待 episode 结束 |
| 在线更新    | 可边运行边学习         |
| 仅使用一步信息 | 效率高，适合实时系统      |
| 不依赖模型   | 类似 MC，但更快更稳定    |

---

## Part 3：TD算法收敛性与 MC 比较

### 收敛性（在策略固定时）：

* 若学习率满足 Robbins-Monro 条件
* 状态空间有限，策略固定

则 $V(s)$ 会收敛到 $V^\pi(s)$

---

### TD vs. MC 比较

| 维度        | TD            | MC          |
| --------- | ------------- | ----------- |
| 是否在线      | ✅ 是           | ❌ 否         |
| 是否支持非终止任务 | ✅ 是           | ❌ 否         |
| 是否有偏      | ✅ 是（因用估计代替真值） | ❌ 无偏        |
| 方差        | ✅ 小（估计稳定）     | ❌ 大（因回报波动大） |
| 收敛速度      | ✅ 快           | ❌ 慢         |
| 使用是否一步    | ✅ 是           | ❌ 否（直到结束）   |

---

## Part 4：SARSA算法（on-policy）

SARSA 是 TD 方法的一个重要扩展，用于估计动作值函数 $Q(s,a)$

---

### 名称来源：

SARSA 表示更新基于：
$(S_t, A_t, R_{t+1}, S_{t+1}, A_{t+1})$

---

### SARSA更新公式：

$$
\boxed{
Q(s_t, a_t) \leftarrow Q(s_t, a_t) + \alpha \left[ r_{t+1} + \gamma Q(s_{t+1}, a_{t+1}) - Q(s_t, a_t) \right]
}
$$

> 这是一种 **on-policy 方法**，即使用当前策略采样的动作更新值函数

---

## Part 5：Expected SARSA 与 n-step SARSA

### Expected SARSA

替代随机动作 $a_{t+1}$ 的单次采样，用动作期望代替：

$$
\boxed{
Q(s_t, a_t) \leftarrow Q(s_t, a_t) + \alpha \left[ r_{t+1} + \gamma \sum_{a'} \pi(a'|s_{t+1}) Q(s_{t+1}, a') - Q(s_t, a_t) \right]
}
$$

优点：减少方差，提高稳定性（尤其在策略是随机策略时）

---

### n-step SARSA

更一般的形式，使用**未来 n 步的回报**：

$$
G_t^{(n)} = r_{t+1} + \gamma r_{t+2} + \dots + \gamma^{n-1} r_{t+n} + \gamma^n Q(s_{t+n}, a_{t+n})
$$

$$
Q(s_t, a_t) \leftarrow Q(s_t, a_t) + \alpha \left[ G_t^{(n)} - Q(s_t, a_t) \right]
$$

---

## Part 6：Q-learning介绍（off-policy）

### 核心思想：

> 用当前策略行为采样（探索），但更新值函数朝着**最优动作的方向**推进

因此，Q-learning 是 **off-policy 学习**。

---

### Q-learning 更新公式：

$$
\boxed{
Q(s_t, a_t) \leftarrow Q(s_t, a_t) + \alpha \left[ r_{t+1} + \gamma \max_{a'} Q(s_{t+1}, a') - Q(s_t, a_t) \right]
}
$$

* 与 SARSA 的区别：SARSA 用实际采样的 $a_{t+1}$，而 Q-learning 用 **max over a'**

---

### On-policy vs Off-policy 比较：

| 维度    | SARSA (on-policy) | Q-learning (off-policy)        |
| ----- | ----------------- | ------------------------------ |
| 更新用动作 | $a_{t+1}$         | $\arg\max_{a'} Q(s_{t+1}, a')$ |
| 稳定性   | 更稳定，适用于危险探索环境     | 更高效，但更易震荡                      |
| 探索方式  | ε-greedy 同策略      | ε-greedy 仅用于采样，学习方向是贪婪         |

---

## Part 7：Q-learning 伪代码与例子

```python
# Q-learning
Initialize Q(s,a) arbitrarily
For each episode:
    Initialize s
    For each step of episode:
        Choose a from s using ε-greedy policy
        Take action a, observe r, s'
        Q(s,a) ← Q(s,a) + α [r + γ max_a' Q(s',a') - Q(s,a)]
        s ← s'
```

---

### 示例：

假设：

* 当前 $Q(s=1, a=left) = 2.0$
* 下一状态 $s'=2$，$Q(s'=2, a') = [1.0, 3.0]$
* $r=1$, $\gamma=0.9$, $\alpha=0.1$

更新：

$$
\text{TD目标} = 1 + 0.9 \times 3.0 = 3.7
\Rightarrow Q(1, left) \leftarrow 2 + 0.1 \times (3.7 - 2) = 2.17
$$

---

## Part 8：TD算法统一形式与总结

TD方法可统一表示为：

$$
\boxed{
\text{Target}_t = r_{t+1} + \gamma \cdot \text{Bootstrap Estimate}
}
$$

| 算法             | Bootstrap Estimate     | 策略类型                      |           |
| -------------- | ---------------------- | ------------------------- | --------- |
| TD(0)          | $V(s_{t+1})$           | on-policy                 |           |
| SARSA          | $Q(s_{t+1}, a_{t+1})$  | on-policy                 |           |
| Expected SARSA | (\sum\_a \pi(a         | s\_{t+1}) Q(s\_{t+1}, a)) | on-policy |
| Q-learning     | $\max_a Q(s_{t+1}, a)$ | off-policy                |           |

---

## 本课总结

* **TD方法结合了 MC 与 DP 的优点**，高效且无需模型
* **SARSA vs Q-learning**：一个稳定保守、一个激进高效
* **Expected SARSA**：低方差版本
* **n-step 方法**：权衡偏差与方差，通向 λ-return
